INSTALL THE FOLLOWING PYTHON PACKAGES FIRST BEFORE RUNNING THE PROGRAM

1) Numpy
2) NNFS - for the Spiral dataset
3) scikit-learn - for the iris dataset

In [1]:
# Library imports
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

Create classes for modularity

In [2]:
# Hidden Layers
# Dense
class Layer_Dense:
    # Layer initialization
    # randomly initialize weights and set biases to zero
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))


    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs, weight and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass/Backpropagation
    def backward(self, dvalues):
        # Gradients on parameters:
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)


In [3]:
# Activation Functions
# Included here are the functions for both the forward and backward pass

# Linear
class ActivationLinear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

# Sigmoid
class ActivationSigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))

# TanH
class ActivationTanH:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)

# ReLU
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the original values first
        self.dinputs = dvalues.copy()
    
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # Remember the inputs values
        self.inputs = inputs

        # Get the unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):

            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate the sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [4]:
# Loss functions

class Loss:
    # Calculate the data and regularization losses
    # Given the model output and grou truth/target values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate the mean loss
        data_loss = np.mean(sample_losses)
        # Return the mean loss
        return data_loss

# MSE
class Loss_MSE:
    def forward(self, y_pred, y_true):
        # Calculate Mean Squared Error
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        # Gradient of MSE loss
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        self.dinputs = -2 * (y_true - y_pred) / outputs
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        # Clip predictions
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Calculate Binary Cross Entropy
        return -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

    def backward(self, y_pred, y_true):
        # Gradient of BCE loss
        samples = y_true.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        self.dinputs = - (y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = y_pred.shape[0]

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values
        # Only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # Use the first sample to count them
        labels = len(dvalues[0])

        # Check if labels are sparse, turn them into one-hot vector values
        # the eye function creates a 2D array with ones on the diagonal and zeros elsewhere
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate the gradient
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


<!-- Star -->

In [5]:
# Start of Optimizers

class Optimizer_SGD:
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0, adaptive=False):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
        self.adaptive = adaptive
        self.epsilon = 1e-7
        self.cache = {}

    # Decay update (happens before FP/BP)
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1.0 / (1.0 + self.decay * self.iterations))

    # Parameter update
    def update_params(self, layer):
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.weight_momentums = weight_updates
            layer.bias_momentums = bias_updates
        elif self.adaptive:
            if layer not in self.cache:
                self.cache[layer] = {'w': np.zeros_like(layer.weights), 'b': np.zeros_like(layer.biases)}
            self.cache[layer]['w'] += layer.dweights ** 2
            self.cache[layer]['b'] += layer.dbiases ** 2
            weight_updates = -self.current_learning_rate * layer.dweights / (np.sqrt(self.cache[layer]['w']) + self.epsilon)
            bias_updates = -self.current_learning_rate * layer.dbiases / (np.sqrt(self.cache[layer]['b']) + self.epsilon)
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1


Use most of the classes to create a functioning neural network, capable of performing a forward and backward pass

We can use a sample dataset from the Spiral module.  

We can also use the IRIS dataset.

In [6]:
# Spiral Data
import nnfs
from nnfs.datasets import spiral_data

# Create the dataset
X, y = spiral_data(samples = 100, classes = 3)

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [7]:
# Iris Dataset
# From the scikit-learn library
# from sklearn.datasets import load_iris
# iris = load_iris()
# X = iris.data # Features
# y = iris.target # Target labels

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [8]:
# Neural Network initialization
# Create a Dense Layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)

# Make sure you check the shape of the features, in order to adjust the input size of the first layer
# dense1 = Layer_Dense(4, 3)

# Create a ReLU activation for the first Dense layer
activation1 = Activation_ReLU()

# Create a 2nd dense layer with 3 input and 3 output values
dense2 = Layer_Dense(3, 3)

# Create a Softmax activation for the 2nd Dense layer
activation2 = Activation_Softmax()

# Create a loss function
loss_function = Loss_CategoricalCrossEntropy()

# Create the optimizer
optimizer = Optimizer_SGD()

PERFORM ONLY 1 PASS

In [9]:
# Perform a forward pass of our training data
# give the input from the dataset to the first layer
dense1.forward(X)

# Activation function
activation1.forward(dense1.output)

# Pass on the 2nd layer
dense2.forward(activation1.output)

activation2.forward(dense2.output)

# Calculate the loss
loss_function.forward(activation2.output, y)

# Check the model's performance
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)

# Print the accuracy
print('acc:', accuracy)

acc: 0.25666666666666665


In [10]:
# Perform a backward pass of our training data
# From loss to 2nd softmax activation
loss_function.backward(activation2.output, y)
dvalues = loss_function.dinputs # Gradient of the loss w.r.t softmax output

print(dvalues.shape)
# print(dvalues)

# From 2nd softmax to 2nd dense layer
activation2.backward(dvalues)
# From 2nd dense layer to 1st ReLU activation
dense2.backward(activation2.dinputs)

# From 1st ReLU activation to 1st dense layer
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

(300, 3)


In [11]:
# Check the gradient values of the weights and biases of the established layers
print(dense1.dweights)
print(dense1.dbiases)
print(dense2.dweights)
print(dense2.dbiases)


# Update the weights and biases
optimizer.update_params(dense1)
optimizer.update_params(dense2)

[[-1.91211918e-04  2.65791491e-04  4.39628317e-05]
 [ 4.35985127e-04 -1.82538890e-04 -2.37138920e-04]]
[[0.00075152 0.00044259 0.00017912]]
[[ 7.87946530e-05 -2.43050037e-04  1.64255384e-04]
 [-1.21397377e-04  1.64560104e-04 -4.31627274e-05]
 [-3.42942244e-05  1.62518399e-05  1.80423845e-05]]
[[-4.07318094e-06 -3.18870404e-06  7.26188498e-06]]


In [12]:
# 🟩 Training Function (with history tracking)
def train_model(X, y, optimizer, epochs=1000, print_every=100):
    history = {'loss': [], 'accuracy': []}  # 🟩 record loss & accuracy each epoch
    
    for epoch in range(epochs):
        optimizer.pre_update_params()
        
        # Forward Pass
        dense1.forward(X)
        activation1.forward(dense1.output)
        dense2.forward(activation1.output)
        activation2.forward(dense2.output)
        
        # Loss + Accuracy
        loss = np.mean(loss_function.forward(activation2.output, y))
        predictions = np.argmax(activation2.output, axis=1)
        if len(y.shape) == 2:
            y = np.argmax(y, axis=1)
        acc = np.mean(predictions == y)
        
        # 🟩 Store metrics
        history['loss'].append(loss)
        history['accuracy'].append(acc)
        
        # Backpropagation
        loss_function.backward(activation2.output, y)
        activation2.backward(loss_function.dinputs)
        dense2.backward(activation2.dinputs)
        activation1.backward(dense2.dinputs)
        dense1.backward(activation1.dinputs)
        
        # Weight update
        optimizer.update_params(dense1)
        optimizer.update_params(dense2)
        optimizer.post_update_params()
        
        # 🟩 Print every 100 epochs
        if (epoch + 1) % print_every == 0 or epoch == 0:
            print(f"Epoch {epoch+1:4d}/1000 - Loss: {loss:.5f} - Accuracy: {acc:.4f} - LR: {optimizer.current_learning_rate:.5f}")
    
    return history  # 🟩 return results for comparison


In [13]:
# ===============================
# 3️⃣ TRAINING EXECUTION SECTION
# ===============================

# Create optimizer instances
opt_sgd = Optimizer_SGD(learning_rate=1.0, decay=1e-3)
opt_mom = Optimizer_SGD(learning_rate=1.0, decay=1e-3, momentum=0.9)
opt_ada = Optimizer_SGD(learning_rate=1.0, decay=1e-3, adaptive=True)

# Train with SGD
print("=== SGD ===")
hist_sgd = train_model(X, y, opt_sgd, epochs=1000, print_every=100)

# Train with Momentum
print("\n=== Momentum ===")
hist_mom = train_model(X, y, opt_mom, epochs=1000, print_every=100)

# Train with AdaGrad
print("\n=== AdaGrad ===")
hist_ada = train_model(X, y, opt_ada, epochs=1000, print_every=100)


=== SGD ===
Epoch    1/1000 - Loss: 1.09862 - Accuracy: 0.2567 - LR: 1.00000
Epoch  100/1000 - Loss: 1.09835 - Accuracy: 0.3933 - LR: 0.90992
Epoch  200/1000 - Loss: 1.08339 - Accuracy: 0.4167 - LR: 0.83403
Epoch  300/1000 - Loss: 1.08087 - Accuracy: 0.4233 - LR: 0.76982
Epoch  400/1000 - Loss: 1.08087 - Accuracy: 0.4200 - LR: 0.71480
Epoch  500/1000 - Loss: 1.08087 - Accuracy: 0.4200 - LR: 0.66711
Epoch  600/1000 - Loss: 1.08087 - Accuracy: 0.4200 - LR: 0.62539
Epoch  700/1000 - Loss: 1.08087 - Accuracy: 0.4200 - LR: 0.58858
Epoch  800/1000 - Loss: 1.08087 - Accuracy: 0.4200 - LR: 0.55586
Epoch  900/1000 - Loss: 1.08087 - Accuracy: 0.4200 - LR: 0.52659
Epoch 1000/1000 - Loss: 1.08087 - Accuracy: 0.4200 - LR: 0.50025

=== Momentum ===
Epoch    1/1000 - Loss: 1.08087 - Accuracy: 0.4200 - LR: 1.00000
Epoch  100/1000 - Loss: 1.08087 - Accuracy: 0.4133 - LR: 0.90992
Epoch  200/1000 - Loss: 1.08087 - Accuracy: 0.4233 - LR: 0.83403
Epoch  300/1000 - Loss: 1.08087 - Accuracy: 0.4200 - LR: 0.7

In [14]:
# 🟩 Compare optimizer performance

def find_stabilization_epoch(losses, window=20, tol=1e-4):
    import numpy as np
    for i in range(window, len(losses)):
        prev_avg = np.mean(losses[i-window:i])
        next_avg = np.mean(losses[i-window+1:i+1])
        if abs(next_avg - prev_avg) < tol:
            return i+1
    return None

stab_sgd = find_stabilization_epoch(hist_sgd['loss']) if 'loss' in hist_sgd else 'N/A'
stab_mom = find_stabilization_epoch(hist_mom['loss']) if 'loss' in hist_mom else 'N/A'
stab_ada = find_stabilization_epoch(hist_ada['loss']) if 'loss' in hist_ada else 'N/A'

print(f"\nStabilization Epochs:")
print(f"SGD: {stab_sgd}, Momentum: {stab_mom}, AdaGrad: {stab_ada}")

print(f"\nFinal Accuracy:")
print(f"SGD: {hist_sgd['accuracy'][-1]:.4f}, Momentum: {hist_mom['accuracy'][-1]:.4f}, AdaGrad: {hist_ada['accuracy'][-1]:.4f}")



Stabilization Epochs:
SGD: 21, Momentum: 21, AdaGrad: 25

Final Accuracy:
SGD: 0.4200, Momentum: 0.4200, AdaGrad: 0.3333
